### Install required packages

In [37]:
! pip3 install ollama

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.1-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached pydantic-2.11.3-py3-none-any.whl (443 kB)
Using cached pydantic_core-2.33.1-cp312-cp312-win_amd64.whl (2.0 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.0-py3-none-any.whl (14 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)


In [50]:
! pip3 install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.5 MB 14.7 MB/s eta 0:00:01
   ----------------- ---------------------- 5.0/11.5 MB 13.7 MB/s eta 0:00:01
   --------------------------- ------------ 7.9/11.5 MB 13.9 MB/s eta 0:00:01
   -------------------------------------- - 11.0/11.5 MB 14.0 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 13.8 MB/s eta 0:00:00


### Import required libraries

In [1]:
import os
import pandas as pd
import ollama
from ollama import chat
from ollama import ChatResponse
from PIL import Image
from io import BytesIO 
import base64
from pydantic import BaseModel, Field
from typing import Literal
import json

### Load and prepare dataset

In [2]:
image_list_safe = []
image_list_talking = []
image_list_text = []

        
for safe in os.listdir("C:/Users/rahul/OneDrive/Documents/HK_project/Revitsone-5classes/safe_driving"): 
    if safe.endswith(".png") or safe.endswith(".jpg"): 
        image_list_safe.append("C:/Users/rahul/OneDrive/Documents/HK_project/Revitsone-5classes/safe_driving/"+safe)
        
for talking in os.listdir("C:/Users/rahul/OneDrive/Documents/HK_project/Revitsone-5classes/talking_phone"): 
    if talking.endswith(".png") or talking.endswith(".jpg"): 
        image_list_talking.append("C:/Users/rahul/OneDrive/Documents/HK_project/Revitsone-5classes/talking_phone/"+talking)
        
for text in os.listdir("C:/Users/rahul/OneDrive/Documents/HK_project/Revitsone-5classes/texting_phone"): 
    if text.endswith(".png") or text.endswith(".jpg"): 
        image_list_text.append("C:/Users/rahul/OneDrive/Documents/HK_project/Revitsone-5classes/texting_phone/"+text)

In [3]:
print("Number of samples in (Class = Safe Driving) = " ,len(image_list_safe))
print("Number of samples in (Class = Talking Phone) = " ,len(image_list_talking))
print("Number of samples in (Class = Texting Phone) = " ,len(image_list_text))

Number of samples in (Class = Safe Driving) =  2203
Number of samples in (Class = Talking Phone) =  2169
Number of samples in (Class = Texting Phone) =  2203


In [4]:
# Split data into train, test and valid 

valid_safe = image_list_safe[-100:]

valid_talking = image_list_talking[-100:]

valid_text = image_list_text[-100:]

In [5]:
valid_list = [valid_safe,valid_talking,valid_text]
classes = ["Safe driving","Talking on the phone","Texting on the phone"]

In [6]:
sys_prompt = """
You are a classification bot. You are monitoring car drivers to raise alert when they are distracted while driving.
Your task is to classify the given image into one of the five following classes.
The classes are ["Safe driving", "Talking on the phone", "Texting on the phone", "Other activities"]. 
The definition of each class is given below. 
    1. "Safe driving" : Images show drivers fully attentive to the road, either with both hands on the steering wheel or one hand on the wheel and the other on the gear stick, exemplifying distraction-free driving.
    2. "Talking on the phone" : Drivers are depicted holding their phones to their ears or speaking on the phone while driving. This class helps to identify the distractions associated with phone conversations.
    3. "Texting on the phone" : Images of drivers using mobile phones to type messages or interact with screens. This class addresses one of the primary causes of distracted driving.
Focus on the driver in the image and Choose the best class from the above classes based on the definition. Do not mention anything else.
"""

In [7]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [8]:
fewshot_message = [    
    {
        "role": "system",
        "content": sys_prompt
    }
]
folders = {
    "safe_driving": ["img_8941.jpg","img_9532.jpg","img_54849.jpg","img_101702.jpg","2019-04-2416-54-28.png"],
    "talking_phone": ["2019-04-2416-38-34.png","2019-04-2417-31-59.png","img_1805.jpg","img_5692.jpg","img_5956.jpg"],
    "texting_phone": ["2019-04-2416-51-31.png","img_216.jpg","img_1027.jpg","img_2631.jpg","img_4490.jpg"],
    }
classes = ["Safe driving","Talking on the phone","Texting on the phone"]
for i,folder in enumerate(folders.keys()):
    for file in folders[folder][:1]:
        file_path = "C:/Users/rahul/OneDrive/Documents/HK_project/Revitsone-5classes/{}/{}".format(folder,file)
        few_shot_1 = {
                "role": "user",
                "content": "Choose the best class for below image",
                "images": [encode_image(file_path)]
            }
        few_shot_2 = {
                "role": "assistant",
                "content": '{"label": "'+classes[i]+'"}',
            }
        fewshot_message.append(few_shot_1)
        fewshot_message.append(few_shot_2)

In [9]:
fewshot_message

[{'role': 'system',
  'content': '\nYou are a classification bot. You are monitoring car drivers to raise alert when they are distracted while driving.\nYour task is to classify the given image into one of the five following classes.\nThe classes are ["Safe driving", "Talking on the phone", "Texting on the phone", "Other activities"]. \nThe definition of each class is given below. \n    1. "Safe driving" : Images show drivers fully attentive to the road, either with both hands on the steering wheel or one hand on the wheel and the other on the gear stick, exemplifying distraction-free driving.\n    2. "Talking on the phone" : Drivers are depicted holding their phones to their ears or speaking on the phone while driving. This class helps to identify the distractions associated with phone conversations.\n    3. "Texting on the phone" : Images of drivers using mobile phones to type messages or interact with screens. This class addresses one of the primary causes of distracted driving.\nFo

In [10]:
class DriverAction(BaseModel):
  label: str = Field(
        ..., enum=["Safe driving","Talking on the phone","Texting on the phone"]
    )

In [15]:

eval_df = pd.DataFrame()
for i in range(len(valid_list)):
    for img_path in valid_list[i]:
        en_img = encode_image(img_path)
        inf_shot = {
                "role": "user",
                "content": "Classify the driver's action from below image",
                "images": [en_img]
            }
        message = fewshot_message + [inf_shot]

        retry = 3
        while retry > 0 :
            try:
                response: ChatResponse = chat(model='gemma3:4b', messages=message, format=DriverAction.model_json_schema())
                predicted = json.loads(response['message']['content'])["label"]
                break
            except:
                print("retrying-",retry)
                retry -= 1
        else:
            raise "Error, chat failed"

        groundtruth = classes[i]
        row_dict = {"image_path": [img_path], "ground_truth": [groundtruth], "predicted": [predicted]}
        eval_df = pd.concat([eval_df,pd.DataFrame(row_dict)])


retrying- 3


In [16]:
eval_df

,image_path,ground_truth,predicted
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Safe driving,Safe driving
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Safe driving,Safe driving
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Safe driving,Safe driving
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Safe driving,Safe driving
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Safe driving,Talking on the phone
...,...,...,...
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Texting on the phone,Texting on the phone
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Texting on the phone,Texting on the phone
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Texting on the phone,Texting on the phone
0,C:/Users/rahul/OneDrive/Documents/HK_project/R...,Texting on the phone,Texting on the phone


In [17]:
accuracy = (len(eval_df[eval_df["ground_truth"] == eval_df["predicted"]])/len(eval_df))*100
print("Accuracy = ",accuracy)

Accuracy =  88.66666666666667
